# Домашнее задание к занятию «Функции потерь и оптимизация» обновленное

Цель: изучить применение методов оптимизации для решения задачи классификации

Описание задания:
В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.

## Библиотеки

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.pipeline import make_pipeline  # используем пайплайны для удобства
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [5]:
from sklearn.datasets import load_iris

## 1. Загрузка данных

In [6]:
iris = load_iris()
mask = np .isin(iris.target, [1, 2])  # выбрать только Versicolor и Virginica
X = iris.data[mask]
y = iris.target[mask] - 1             # перекодирование классов: 0 и 1

## 2. Функция сигмоид

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

## 3. Реализация логистической регрессии с методом градиентного спуска

Градиентный спуск

In [8]:

def fit_gradient_descent(X, y, learning_rate=0.01, num_iterations=1000):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)                                  # инициализация весов
    bias = 0                                                       # инициализация смещения

    for _ in range(num_iterations):
        linear_model = np.dot(X, weights) + bias
        y_predicted = sigmoid(linear_model)                        # предсказанная вероятность
        dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))      # градиент весов
        db = (1 / n_samples) * np.sum(y_predicted - y)             # градиент смещения

                                                                # обновление параметров
        weights -= learning_rate * dw
        bias -= learning_rate * db
    
    return weights, bias

## 4. Реализация логистической регрессии с методом RMSProp

In [9]:
def fit_rmsprop(X, y, learning_rate=0.01, num_iterations=1000, beta1=0.9, beta2=0.999, epsilon=1e-8):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0
    m = np.zeros(n_features)                                     # накопленный средний градиент
    v = np.zeros(n_features)                                     # накопленная средняя квадрата градиента

    for _ in range(num_iterations):
        linear_model = np.dot(X, weights) + bias
        y_predicted = sigmoid(linear_model)
        dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
        db = (1 / n_samples) * np.sum(y_predicted - y)

        m = beta1 * m + (1 - beta1) * dw
        v = beta2 * v + (1 - beta2) * (dw ** 2)
        weights -= learning_rate * m / (np.sqrt(v) + epsilon)
        bias -= learning_rate * db

    return weights, bias

## 5. Реализация логистической регрессии с методом Nadam

In [10]:
def fit_nadam(X, y, learning_rate=0.01, num_iterations=1000, beta1=0.9, beta2=0.999, epsilon=1e-8):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0
    m = np.zeros(n_features)
    v = np.zeros(n_features)
    t = 0

    for _ in range(num_iterations):
        t += 1
        linear_model = np.dot(X, weights) + bias
        y_predicted = sigmoid(linear_model)
        dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
        db = (1 / n_samples) * np.sum(y_predicted - y)

        m = beta1 * m + (1 - beta1) * dw
        v = beta2 * v + (1 - beta2) * (dw ** 2)

        m_hat = m / (1 - beta1 ** t)                                            # корректировка первого момента
        v_hat = v / (1 - beta2 ** t)                                            # корректировка второго момента

        weights -= learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + beta1 * m_hat)
        bias -= learning_rate * db

    return weights, bias

## 6. Дополнительные функции

In [11]:
def predict(X, weights, bias):
    linear_model = np.dot(X, weights) + bias
    y_predicted_prob = sigmoid(linear_model)
    return (y_predicted_prob >= 0.5).astype(int)

def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)


## 7. Проверка

Подготовка расчетов

In [12]:
methods = {
    "Gradient Descent": fit_gradient_descent,
    "RMSProp": fit_rmsprop,
    "Nadam": fit_nadam,
}

Обучение моделей и сбор результатов

In [13]:
results = []
for method_name, method_func in methods.items():
    start_time = time.time()
    weights, bias = method_func(X, y)
    end_time = time.time()
    y_pred = predict(X, weights, bias)
    acc = accuracy(y, y_pred)
    elapsed_time = end_time - start_time
    results.append((method_name, acc, elapsed_time))

Вывод результатов в виде таблицы

In [14]:
print("| Метод                | Точность (Accuracy) | Время работы (сек) |")
print("|----------------------|---------------------|--------------------|")
for method_name, acc, elapsed_time in results:
    print(f"| {method_name:20} | {acc:.4f}              | {elapsed_time:.4f}           |")

| Метод                | Точность (Accuracy) | Время работы (сек) |
|----------------------|---------------------|--------------------|
| Gradient Descent     | 0.9600              | 0.0084           |
| RMSProp              | 0.9300              | 0.0118           |
| Nadam                | 0.9600              | 0.0138           |


## 8. Выводы

В процессе работы написаны функции для различных методовов реализации логистической регрессии. Ответы выведены в отдельную таблицу. Лучший результат показали Градиентный спуск и Nadam. А учитывая почти в 2 раза меньшие временные затраты, лучшим методом себя обычный градиентный спуск.